# GMOCAT-Enhanced Experiment Runner

Notebook ini digunakan untuk menjalankan eksperimen GMOCAT yang telah dimodifikasi dengan fitur:
1. Coverage-Aware Reward
2. Uncertainty-Based Termination
3. Adaptive Diversity Weight

Tujuannya adalah mencapai Concept Coverage >= 70% pada dataset `dbekt22`.

In [1]:
import sys
import os
import torch
import numpy as np
import logging

# Tambahkan direktori saat ini ke path agar modul bisa diimport
sys.path.append(os.getcwd())

# Cek device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Menggunakan device: {device}")

Menggunakan device: cpu


In [2]:
# Definisi Argumen (Pengganti argparse)
class Args:
    def __init__(self):
        # Konfigurasi Dasar
        self.seed = 145
        self.environment = "GCATEnv"
        self.data_path = "./data/"
        self.data_name = "dbekt22" # Target dataset
        self.agent = "GCATAgent"
        self.FA = "GCAT"
        self.CDM = "NCD"
        
        # Pengaturan Testing
        self.T = 20 # Panjang tes (steps)
        self.ST = [5, 10, 20] # Step evaluasi
        self.student_ids = [0] # Dummy
        self.target_concepts = [0] # Dummy
        
        # Hyperparameters Training
        self.gpu_no = "0"
        self.device = device
        self.learning_rate = 0.001
        self.training_epoch = 5 # Set ke angka lebih tinggi (misal 20) untuk hasil maksimal
        self.train_bs = 64      # Kurangi jika memory error (misal 32)
        self.test_bs = 1024     
        self.batch = 128
        
        # Parameter Model & RL
        self.cdm_lr = 0.02
        self.cdm_epoch = 2 # Epoch untuk update CDM per step
        self.cdm_bs = 128
        self.gamma = 0.5
        self.latent_factor = 256
        self.n_block = 1
        self.graph_block = 1
        self.n_head = 1
        self.dropout_rate = 0.1
        self.policy_epoch = 1
        self.morl_weights = [1, 1, 1] # Bobot [Accuracy, Diversity, Novelty]
        self.emb_dim = 128
        self.use_graph = True
        self.use_attention = True
        self.store_action = False

    def __str__(self):
        return str(self.__dict__)

args = Args()
print("Konfigurasi dimuat.")

Konfigurasi dimuat.


In [4]:
from launch_gcat import construct_local_map
from util import get_objects, set_global_seeds
import envs as all_envs
import agents as all_agents
import function as all_FA

# Set seed
set_global_seeds(args.seed)
if args.device == 'cuda':
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu_no)

# Setup Logger (agar output muncul di notebook)
logger = logging.getLogger(f'{args.FA}')
if not logger.handlers:
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    # StreamHandler ke stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)
    logger.addHandler(ch)

# Load Graph
print(f"Loading Graph Data untuk {args.data_name}...")

# Setup Environment
print("Setup Environment...")
# Setup Environment
print("Setup Environment...")
envs = get_objects(all_envs)
env = envs[args.environment](args)

# Update args dengan info dari env
args.user_num = env.user_num
args.item_num = env.item_num
args.know_num = env.know_num

print(f"Environment siap. User: {args.user_num}, Item: {args.item_num}, Concept: {args.know_num}")

# Load Graph (setelah info env tersedia)
print(f"Loading Graph Data untuk {args.data_name}...")
local_map = construct_local_map(args, path=f'graph_data/{args.data_name}/')

# Setup Model (Function Approximation)
print("Membangun Model GCAT...")
nets = get_objects(all_FA)
fa = nets[args.FA].create_model(args, local_map)

# Setup Agent
print("Inisialisasi Agen...")
agents = get_objects(all_agents)
agent = agents[args.agent](env, fa, args)
print("Sistem siap dilatih.")

Loading Graph Data untuk dbekt22...
Setup Environment...
Setup Environment...
2026-02-19 09:31:21,673 - train/val/test size: 793, 99,99
2026-02-19 09:31:21,705 - models/dbekt22/NCD_dbekt22_20.pt
Environment siap. User: 991, Item: 213, Concept: 93
Loading Graph Data untuk dbekt22...
Membangun Model GCAT...
Inisialisasi Agen...
Sistem siap dilatih.


In [5]:
# Jalankan Training
# Proses ini mungkin memakan waktu tergantung spesifikasi hardware
print("Memulai Training...")
agent.train()
print("Training Selesai.")

Memulai Training...


/usr/local/python/3.12.1/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


: 

In [ ]:
# Analisis Hasil & Visualisasi
import matplotlib.pyplot as plt
from analyze_results import parse_log
import glob

# Fungsi untuk membaca coverage dari memori agent (jika tersedia) atau log terbaru
# Karena kita menjalankan di notebook, kita bisa akses agent.all_cov jika dimodifikasi untuk persist,
# tapi defaultnya agent me-reset all_cov. Kita cari log file yang baru saja dibuat.

log_dir = f'baseline_log/{args.data_name}'
if os.path.exists(log_dir):
    list_of_files = glob.glob(f'{log_dir}/*.txt')
    if list_of_files:
        latest_file = max(list_of_files, key=os.path.getctime)
        print(f"Menganalisis log: {latest_file}")

        data = parse_log(latest_file)

        if data:
            plt.figure(figsize=(10, 6))
            plt.plot(range(1, len(data) + 1), data, marker='o', label='Concept Coverage')
            plt.title(f'Concept Coverage Curve ({args.data_name})')
            plt.xlabel('Step')
            plt.ylabel('Coverage Ratio')
            plt.grid(True)
            plt.legend()
            plt.show()
            print(f"Final Coverage: {data[-1]*100:.2f}%")
        else:
            print("Data coverage tidak ditemukan di log (mungkin format berbeda atau training belum selesai).")
    else:
        print("Belum ada file log.")
else:
    print("Direktori log tidak ditemukan.")